# AirNow API Historical Data Pull Python Script
The below Python script will detail how to pull historical data from the AirNow API and write the data to a BigQuery table.

In [1]:
# Authenticate GCP script
# Install packages
from google.cloud import bigquery
from google.oauth2 import service_account

# Service account JSON key file path
service_acct_path = r'SERVICE_ACCOUNT_PATH'

credentials = service_account.Credentials.from_service_account_file(service_acct_path)

# Initialize BigQuery client
project_id = 'key-polymer-434418-m6'
bq_client = bigquery.Client(credentials=credentials, project=project_id)

# Authentication check by listing datasets in the project
try:
    datasets = list(bq_client.list_datasets())  # Get list of datasets in the project
    if datasets:
        print(f"Successfully authenticated! Found {len(datasets)} dataset(s) in project {project_id}.")
        for dataset in datasets:
            print(f"- {dataset.dataset_id}")
    else:
        print(f"Successfully authenticated, but no datasets found in project {project_id}.")
except Exception as e:
    print(f"Failed to authenticate or access BigQuery: {e}")

Successfully authenticated! Found 3 dataset(s) in project key-polymer-434418-m6.
- airnow_analytics
- airnow_raw
- airnow_staging


In [2]:
# Iterate through dates for AirNow API Query script
# Install Packages
import pandas as pd
import requests
from datetime import datetime, timedelta

API_KEY = 'API_KEY'
BASE_URL = 'https://www.airnowapi.org/aq/observation/zipCode/historical/'

# Function to retrieve AirNow data
def get_air_quality_data(start_date, end_date, zip_code='96150'):
    # Convert the input dates to datetime objects
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

    # List to store all data
    all_data = []

    # Base URL for the AirNow API
    url_hist = 'https://www.airnowapi.org/aq/observation/zipCode/historical/'

    # Loop through the date range
    current_date = start_date
    while current_date <= end_date:
        # Format the current date as required by the API
        date_str = current_date.strftime('%Y-%m-%dT00-0000')

        # Parameters for the API request
        params_hist = {
            'format': 'application/json',
            'zipCode': zip_code,  # Default to South Lake Tahoe, CA
            'date': date_str,
            'distance': '25',
            'API_KEY': API_KEY
        }

        # Make the request
        response_hist = requests.get(url_hist, params=params_hist)

        # Check if the request was successful
        if response_hist.status_code == 200:
            data_hist = response_hist.json()

            # Add the data to the list if any records were returned
            if data_hist:
                all_data.extend(data_hist)
        else:
            print(f"Failed to retrieve data for {date_str}. Status code: {response_hist.status_code}")

        # Move to the next day
        current_date += timedelta(days=1)

    # Convert the list of data into a DataFrame
    df_hist = pd.DataFrame(all_data)

    # Return the DataFrame
    return df_hist

# Example usage:
# Define the start and end dates
#start_date = '2024-01-01'  # Example start date
#end_date = '2024-01-02'    # Example end date

# Call the function to retrieve data for the specified date range
#df_hist = get_air_quality_data(start_date, end_date)

# Display the DataFrame
#df_hist

In [3]:
# Transform all columns to string except for 'DateObserved' (further transformations will be done with dbt)
def transform_date_data(df):
    # Convert all object and float64 columns to strings to avoid issues
    df = df.astype(str)
    # Convert specific columns to strings and handle NaNs
    df = df.fillna('').astype(str)
    df['DateObserved'] = pd.to_datetime(df['DateObserved'])
    return df

#df_transformed = transform_date_data(df_hist)
#df_transformed

In [4]:
# Create BigQuery table with Partitioning
def create_bq_table(client, dataset_id, table_id):
    dataset_ref = client.dataset(dataset_id)
    schema = [
        bigquery.SchemaField('DateObserved', 'TIMESTAMP'),
        bigquery.SchemaField('HourObserved', 'STRING'),
        bigquery.SchemaField('LocalTimeZone', 'STRING'),
        bigquery.SchemaField('ReportingArea', 'STRING'),
        bigquery.SchemaField('StateCode', 'STRING'),
        bigquery.SchemaField('Latitude', 'STRING'),
        bigquery.SchemaField('Longitude', 'STRING'),
        bigquery.SchemaField('ParameterName', 'STRING'),
        bigquery.SchemaField('AQI', 'STRING'),
        bigquery.SchemaField('Category', 'STRING')
    ]
    
    table = bigquery.Table(dataset_ref.table(table_id), schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
        field='DateObserved'
    )
    client.create_table(table, exists_ok=True)
    print(f"Table {table_id} created or already exists.")

# Example:
#create_bq_table(bq_client, 'your_dataset', 'airnow_hist_raw')

In [5]:
# Load DataFrame (historical data) into BigQuery
def load_data_to_bq(client, df, dataset_id, table_id):
    table_ref = client.dataset(dataset_id).table(table_id)
    job = client.load_table_from_dataframe(df, table_ref)
    job.result()  # Wait for the job to complete
    print(f"Loaded {df.shape[0]} rows into {table_id}.")

# Example:
#load_data_to_bq(bq_client, df_transformed, 'your_dataset', 'airnow_hist_raw')

In [6]:
# Put all the functions together to create the BigQuery table and load the historical data
# First data pull will be YTD (as of 2024-09-02)
start_date = '2024-09-02'
end_date = '2024-10-06'

# Call the function to retrieve data for the specified date range
df = get_air_quality_data(start_date, end_date)

# Transform the data
df_transformed = transform_date_data(df)

# Create BigQuery table (only need to use if the table does not exist)
create_bq_table(bq_client, 'airnow_raw', 'airnow_hist_raw')

# Load data into BigQuery ('airnow_hist_raw')
load_data_to_bq(bq_client, df_transformed, 'airnow_raw', 'airnow_hist_raw')

Table airnow_hist_raw created or already exists.
Loaded 105 rows into airnow_hist_raw.
